<a href="https://colab.research.google.com/github/pds2526/course/blob/main/10_NLP_with_huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div class='bar_title'></div>

*Practical Data Science*

# NLP with 🤗 Transformers

Gunther Gust<br>
Chair for Enterprise AI <br>
Center for Artificial Intelligence and Data Science (CAIDAS)

<img src="https://github.com/GuntherGust/tds2_data/blob/main/images/d3.png?raw=true" style="width:20%; float:left;" />

<img src="https://github.com/GuntherGust/tds2_data/blob/main/images/CAIDASlogo.png?raw=true" style="width:20%; float:left;" />

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Summary-of-the-tasks" data-toc-modified-id="Summary-of-the-tasks-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Summary of the tasks</a></span><ul class="toc-item"><li><span><a href="#Sequence-Classification" data-toc-modified-id="Sequence-Classification-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Sequence Classification</a></span></li><li><span><a href="#Extractive-Question-Answering" data-toc-modified-id="Extractive-Question-Answering-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Extractive Question Answering</a></span></li><li><span><a href="#Language-Modeling" data-toc-modified-id="Language-Modeling-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Language Modeling</a></span><ul class="toc-item"><li><span><a href="#Masked-Language-Modeling" data-toc-modified-id="Masked-Language-Modeling-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Masked Language Modeling</a></span></li><li><span><a href="#Causal-Language-Modeling" data-toc-modified-id="Causal-Language-Modeling-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Causal Language Modeling</a></span></li></ul></li><li><span><a href="#Text-Generation" data-toc-modified-id="Text-Generation-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Text Generation</a></span></li><li><span><a href="#Named-Entity-Recognition" data-toc-modified-id="Named-Entity-Recognition-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Named Entity Recognition</a></span></li><li><span><a href="#Summarization" data-toc-modified-id="Summarization-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Summarization</a></span></li><li><span><a href="#Translation" data-toc-modified-id="Translation-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Translation</a></span></li></ul></li><li><span><a href="#Fine-tuning-a-pretrained-model" data-toc-modified-id="Fine-tuning-a-pretrained-model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Fine-tuning a pretrained model</a></span><ul class="toc-item"><li><span><a href="#Preparing-the-datasets" data-toc-modified-id="Preparing-the-datasets-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Preparing the datasets</a></span></li><li><span><a href="#Fine-tuning-in-PyTorch-with-the-Trainer-API" data-toc-modified-id="Fine-tuning-in-PyTorch-with-the-Trainer-API-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Fine-tuning in PyTorch with the Trainer API</a></span></li><li><span><a href="#Additional-resources" data-toc-modified-id="Additional-resources-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Additional resources</a></span></li></ul></li></ul></div>

__Credits for this lecuture__

- https://huggingface.co/docs/transformers/task_summary
- https://huggingface.co/docs/transformers/training

In [6]:
#!pip install --force-reinstall "accelerate==0.26.1" "peft==0.10.0" "transformers==4.36.2" "sentence-transformers==2.7.0"


In [7]:
import warnings
warnings.filterwarnings('ignore') # Ignore warnings for clarity

# Install on colab
# Update accelerate and transformers to be compatible with sentence-transformers.
# This changes the original explicit downgrade for Trainer API compatibility.
#!pip install accelerate>=0.28.0
#!pip install -U transformers # Install the latest transformers
#!pip install datasets
#!pip install -U sentence-transformers # Ensure sentence-transformers is explicitly installed and updated to be compatible

## Summary of the tasks

This section shows the **most frequent use-cases** when using the huggingface library. The models available allow for many different configurations and a great versatility in use-cases. The most simple ones are presented here, showcasing usage for
tasks such as question answering, sequence classification, named entity recognition and others.

These following examples leverage __auto-models__, which are classes that will instantiate a model according to a given checkpoint, __automatically__ selecting the __correct model architecture.__ Please check the `AutoModel` [documentation](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html) for more information.

In order for a model to perform well on a task, it must be loaded from a __checkpoint corresponding__ to that __task.__ These
checkpoints are usually pre-trained on a __large corpus of data__ and fine-tuned on a __specific task.__

> **NOTE:** A __model checkpoint__ in deep learning is similar to saving a game progress in a video game. Just as you save your progress to avoid losing your achievements and to resume playing from the same spot later, a model checkpoint saves the state of a neural network at a particular point during training. <br>
> A checkpoint typically includes various elements such as the model’s __architecture__ (how the neural network is structured), the model’s __weight parameters__ (the learned aspects of the model), and the __optimizer state__ (information about the algorithm used to update the weights).

In order to do an inference on a task, several mechanisms are made available by the library:

- __Pipelines:__ very easy-to-use abstractions, which require as little as two lines of code.
- __Direct model use:__ Less abstractions, but more flexibility and power via a direct __access to a tokenizer__
  (PyTorch/TensorFlow) and full inference capacity.

> **NOTE:** All tasks presented here leverage pre-trained checkpoints that were fine-tuned on specific tasks. Loading a
> checkpoint that was not fine-tuned on a specific task would load only the base transformer layers and not the
> additional head that is used for the task, initializing the weights of that head randomly. This would produce random output. <br><br>
> In the __second section__ of this notebook, we show how such fine-tuning a custom head is done.

### Sequence Classification

Here is an example of using pipelines to do __sentiment analysis:__ identifying if a sequence is positive or negative. It
leverages a fine-tuned model on sst2, which is a General Language Understanding Evaluation (GLUE) task.

This returns a label ("POSITIVE" or "NEGATIVE") alongside a score, as follows:

In [8]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis")
result = classifier("I hate you")[0]
print(f"label: {result['label']}, with score: {round(result['score'], 4)}")
result = classifier("I love you")[0]
print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


label: NEGATIVE, with score: 0.9991
label: POSITIVE, with score: 0.9999


#### Model + Tokenizer

Here is an example of doing a sequence classification using a model to determine if two sequences are __paraphrases of
each other.__


In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# "bert-base-cased-finetuned-mrpc" is a version of BERT that has been fine-tuned on the
# Microsoft Research Paraphrase Corpus (MRPC) dataset
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased-finetuned-mrpc")
classes = ["not paraphrase", "is paraphrase"]
sequence_0 = "The company HuggingFace is based in New York City"
sequence_1 = "Apples are especially bad for your health"
sequence_2 = "HuggingFace's headquarters are situated in Manhattan"
# The tokenizer will automatically add any model specific separators (i.e. <CLS> and <SEP>) and tokens to
# the sequence, as well as compute the attention masks.
paraphrase = tokenizer(sequence_0, sequence_2, return_tensors="pt")
not_paraphrase = tokenizer(sequence_0, sequence_1, return_tensors="pt")
paraphrase_classification_logits = model(**paraphrase).logits
not_paraphrase_classification_logits = model(**not_paraphrase).logits
paraphrase_results = torch.softmax(paraphrase_classification_logits, dim=1).tolist()[0]
not_paraphrase_results = torch.softmax(not_paraphrase_classification_logits, dim=1).tolist()[0]
# Should be paraphrase
for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(paraphrase_results[i] * 100))}%")
# Should not be paraphrase
for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(not_paraphrase_results[i] * 100))}%")

not paraphrase: 10%
is paraphrase: 90%
not paraphrase: 94%
is paraphrase: 6%


The model "bert-base-cased-finetuned-mrpc" is a version of BERT that has been fine-tuned on the Microsoft Research Paraphrase Corpus (MRPC) dataset, a dataset specifically designed for the task of paraphrase identification.


The process is the following:

1. Instantiate a tokenizer and a model from the checkpoint name. The model is identified as a BERT model and loads it
   with the weights stored in the checkpoint.
2. Build a sequence from the two sentences, with the correct model-specific separators, token type ids and attention
   masks (which will be created automatically by the tokenizer).
3. Pass this sequence through the model so that it is classified in one of the two available classes: 0 (not a
   paraphrase) and 1 (is a paraphrase).
4. Compute the softmax of the result to get probabilities over the classes.
5. Print the results.

### Sentence embeddings

**Sentence Transformers** provide a simple way to convert entire sentences (or paragraphs/documents) into **dense vector embeddings**.  These embeddings capture **semantic meaning**, so sentences with similar meaning end up __close together in vector space.__

(The previous paraphrase detection model learns to classify paraphrases, but it does not reorganize BERT’s embedding space to represent semantic similarity — so the classifier works, but measuring cosine-based similarity will only work suboptimally at best.)


Traditional word embeddings (like Word2Vec or GloVe) produce vectors for **individual words**, but Sentence Transformers extend this idea to **full sentences**. This makes them ideal for:

- Semantic search  
- Clustering documents  
- Detecting paraphrases  
- Re-ranking search results  
- Recommendation systems  

#### How a Sentence Transformer Works (Intuition)

A Sentence Transformer takes a sentence as input and passes it through a **Transformer encoder** (e.g., BERT, RoBERTa, MPNet). Then it applies a **pooling operation** (often mean pooling) to convert the token-level outputs into **one single vector**.

Steps:
1. Tokenize the sentence  
2. Feed tokens into a transformer model  
3. Extract the contextual embeddings (one vector per token)  
4. Pool them (typically average)  
5. Output a single embedding vector, e.g. 768 dimensions  

#### How Sentence Transformers Are Trained

Sentence Transformers are built __on top of models__ like BERT, RoBERTa, or MPNet, but they are **fine-tuned with a special training setup** so that entire sentences can be compared.

##### **Siamese / Triplet Network Architecture**
Instead of feeding one sentence at a time, the model processes **pairs or triplets** of sentences:

- **Positive pair:** sentences with similar meaning  
- **Negative pair:** sentences with different meaning  

The model is trained so that:

- embeddings of similar sentences are **close together**  
- embeddings of dissimilar sentences are **far apart**

Common loss functions are cosine similarity loss, triplet loss, contrastive loss, and multiple negative ranking loss.

##### **Typical Training Data**
Models are fine-tuned on datasets such as:

- Paraphrase pairs (e.g., STS benchmark, Quora duplicates)  
- Natural language inference (entailment, contradiction)  
- Question–answer pairs  
- Other large data sources from the web
    - e.g., Wikipedia sentence and its page title

These provide natural positive/negative relationships.

### Measuring semantic similarity using cosine similartiy

Cosine similarity is a measure of how **similar two vectors are**, based on the **angle** between them rather than their magnitude.  


$\displaystyle
\text{cos_sim}(\mathbf{a}, \mathbf{b}) =
\frac{\mathbf{a} \cdot \mathbf{b}}{\lVert \mathbf{a} \rVert \, \lVert \mathbf{b} \rVert}
$

Where:

- $\mathbf{a} \cdot \mathbf{b} = \sum_{i=1}^{n} a_i b_i$ is the **dot product**
- $\lVert \mathbf{a} \rVert = \sqrt{\sum_{i=1}^{n} a_i^2}$ is the **magnitude** (length) of vector $\mathbf{a}$
- The result is in the range

  $$0 \le \text{cos_sim} \le 1$$
  
Cases:
  - **1** → vectors point in the same direction (very similar)  
  - **0** → vectors are orthogonal (no similarity)  





#### Usefulness of cosine similarity for text analysis

- Document vectors often differ in length (e.g., short vs. long documents).  
- Cosine similarity ignores magnitude and focuses only on direction → **great for comparing text content**.  
- Common use cases:
  - Document search (“find texts most similar to this query”)  
  - Clustering documents  
  - Comparing word embeddings  

In [10]:
from sentence_transformers import SentenceTransformer, util

# A strong general-purpose embedding model
model = SentenceTransformer("all-mpnet-base-v2")

sentences = [
    "The company HuggingFace is based in New York City.",
    "Apples are especially bad for your health.",
    "HuggingFace's headquarters are situated in Manhattan."
]

# Compute embeddings
embeddings = model.encode(sentences, convert_to_tensor=True)

# Compute pairwise similarity
sim_0_1 = util.cos_sim(embeddings[0], embeddings[1]).item()
sim_0_2 = util.cos_sim(embeddings[0], embeddings[2]).item()

print("Similarity between sentence 0 and 1:", round(sim_0_1, 3))
print("Similarity between sentence 0 and 2:", round(sim_0_2, 3))

# A simple interpretation
def interpret(score):
    if score > 0.7: return "highly similar"
    if score > 0.4: return "somewhat related"
    return "not similar"

print("Sentence 0 vs 1:", interpret(sim_0_1))
print("Sentence 0 vs 2:", interpret(sim_0_2))

Similarity between sentence 0 and 1: 0.003
Similarity between sentence 0 and 2: 0.859
Sentence 0 vs 1: not similar
Sentence 0 vs 2: highly similar


### Extractive Question Answering

Extractive Question Answering is the task of
* extracting an answer
* from a __text__
* given a __question.__ <br>

An example of a question answering dataset is the SQuAD dataset, which is entirely based on that task.

Here is an __example__ of using pipelines to do question answering: extracting an answer from a text given a question. It
leverages a fine-tuned model on the [__Stanford Question Answering Dataset__ (SQuAD)](https://rajpurkar.github.io/SQuAD-explorer/).

In [11]:
from transformers import pipeline
question_answerer = pipeline("question-answering")

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


The following returns an
* __answer__ extracted from the text,
* a __confidence score__,
* alongside __"start" and "end" values,__ which are the positions of the extracted answer in the text.

In [12]:
context = r"""
Extractive Question Answering is the task of extracting an answer from a text given a question. An example of a
question answering dataset is the SQuAD dataset, which is entirely based on that task. If you would like to fine-tune
a model on a SQuAD task, you may leverage the examples/pytorch/question-answering/run_squad.py script.
"""

result = question_answerer(question="What is extractive question answering?", context=context)
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")
result = question_answerer(question="What is a good example of a question answering dataset?", context=context)
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Answer: 'the task of extracting an answer from a text given a question', score: 0.6177, start: 34, end: 95
Answer: 'SQuAD dataset', score: 0.5153, start: 147, end: 160


#### Model + Tokenizer

Here is an example of question answering using a model and a tokenizer.

In [13]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
text = r"""
🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose
architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural
Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between
TensorFlow 2.0 and PyTorch.
"""
questions = [
    "How many pretrained models are available in 🤗 Transformers?",
    "What does 🤗 Transformers provide?",
    "🤗 Transformers provides interoperability between which frameworks?",
]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
for question in questions:
    inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    # Get the most likely beginning of answer with the argmax of the score
    answer_start = torch.argmax(answer_start_scores)
    # Get the most likely end of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    print(f"Question: {question}")
    print(f"Answer: {answer}")

Question: How many pretrained models are available in 🤗 Transformers?
Answer: over 32 +
Question: What does 🤗 Transformers provide?
Answer: general - purpose architectures
Question: 🤗 Transformers provides interoperability between which frameworks?
Answer: tensorflow 2. 0 and pytorch


The process is the following:

1. Instantiate a tokenizer and a model from the checkpoint name. The model is identified as a BERT model and loads it
   with the weights stored in the checkpoint.
2. Define a text and a few questions.
3. Iterate over the questions and build a sequence from the text and the current question, with the correct
   model-specific separators token type ids and attention masks.
4. Pass this sequence through the model. This outputs a range of scores across the entire sequence tokens (question and
   text), for both the start and end positions.
5. Compute the softmax of the result to get probabilities over the tokens.
6. Fetch the tokens from the identified start and stop values, convert those tokens to a string.
7. Print the results.

### Language Modeling

Language modeling is the task of fitting a model to a corpus, which can be domain specific. All popular transformer-based models are trained using a variant of language modeling,
* e.g. BERT with __masked language modeling,__
* GPT-2 with __causal language modeling.__

Language modeling can be useful outside of pretraining as well, for example to shift the __model distribution to be
domain-specific__.

For example, [this model LysandreJik/arxiv-nlp](https://huggingface.co/lysandre/arxiv-nlp) is trained over a very large corpus, and then fine-tuned to scientific papers.

#### Masked Language Modeling

Masked language modeling is the task of masking tokens in a sequence with a masking token, and prompting the model to
fill that mask with an appropriate token. This allows the model to attend to both the __right__ context (tokens on the
right of the mask) and the __left context__ (tokens on the left of the mask).

Such a training creates a strong basis for __downstream tasks requiring bi-directional context__, such as question answering, see [Lewis, Lui, Goyal et al.](https://arxiv.org/abs/1910.13461), part 4.2).

Here is an example of using pipelines to replace a mask from a sequence:

In [15]:
from transformers import pipeline
unmasker = pipeline("fill-mask")

No model was supplied, defaulted to distilbert/distilroberta-base and revision fb53ab8 (https://huggingface.co/distilbert/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert/distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


This outputs the __sequences with the mask filled,__ the confidence score, and the token id in the tokenizer vocabulary. (It uses [F-strings](https://realpython.com/python-f-strings/) and [pprint](https://docs.python.org/3/library/pprint.html) ("pretty print"), which make string formatting and printing more conventient.)

In [16]:
from pprint import pprint

pprint(unmasker(f"HuggingFace is creating a {unmasker.tokenizer.mask_token} that the community uses to solve NLP tasks."))

[{'score': 0.17927402257919312,
  'sequence': 'HuggingFace is creating a tool that the community uses to solve '
              'NLP tasks.',
  'token': 3944,
  'token_str': ' tool'},
 {'score': 0.11349376291036606,
  'sequence': 'HuggingFace is creating a framework that the community uses to '
              'solve NLP tasks.',
  'token': 7208,
  'token_str': ' framework'},
 {'score': 0.05243528261780739,
  'sequence': 'HuggingFace is creating a library that the community uses to '
              'solve NLP tasks.',
  'token': 5560,
  'token_str': ' library'},
 {'score': 0.03493509441614151,
  'sequence': 'HuggingFace is creating a database that the community uses to '
              'solve NLP tasks.',
  'token': 8503,
  'token_str': ' database'},
 {'score': 0.02860235795378685,
  'sequence': 'HuggingFace is creating a prototype that the community uses to '
              'solve NLP tasks.',
  'token': 17715,
  'token_str': ' prototype'}]


#### Model + Tokenizer


Here is an example of doing masked language modeling using a model and a tokenizer. The process is the following:

1. Instantiate a tokenizer and a model from the checkpoint name. The model is identified as a DistilBERT model and
   loads it with the weights stored in the checkpoint.
2. Define a sequence with a masked token, placing the `tokenizer.mask_token` instead of a word.
3. Encode that sequence into a list of IDs and find the position of the masked token in that list.
4. Retrieve the predictions at the index of the mask token: this tensor has the same size as the vocabulary, and the
   values are the scores attributed to each token. The model gives higher score to tokens it deems probable in that
   context.
5. Retrieve the top 5 tokens using the PyTorch `topk` or TensorFlow `top_k` methods.
6. Replace the mask token by the tokens and print the results

In [17]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")
model = AutoModelForMaskedLM.from_pretrained("distilbert-base-cased")
sequence = "Distilled models are smaller than the models they mimic. Using them instead of the large " \
    f"versions would help {tokenizer.mask_token} our carbon footprint."
inputs = tokenizer(sequence, return_tensors="pt")
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
token_logits = model(**inputs).logits
mask_token_logits = token_logits[0, mask_token_index, :]
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()
for token in top_5_tokens:
    print(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Distilled models are smaller than the models they mimic. Using them instead of the large versions would help reduce our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help increase our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help decrease our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help offset our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help improve our carbon footprint.


This prints five sequences, with the top 5 tokens predicted by the model.

#### Causal Language Modeling

Causal language modeling is the task of predicting the __token following a sequence of tokens__. In this situation, the
model __only__ attends to the __left context__ (tokens on the left of the mask). Such a training is particularly interesting
for __text generation__ tasks.

Usually, the next token is predicted by sampling from the logits of the last hidden state the model produces from the
input sequence.

Here is an example of using the tokenizer and model and leveraging the `PreTrainedModel.top_k_top_p_filtering` method to sample the __next token__ following an input sequence of tokens.

> __Top-k Filtering:__ The function first applies Top-k filtering by selecting the __'k' most likely next words__ from the vocabulary based on their logit scores. In your example, top_k=50 means that the model will only consider the top 50 words as candidates for the next word in the sequence. This helps in reducing the choice set to the most probable words, enhancing coherence.

>__Top-p (Nucleus) Filtering:__ After applying Top-k filtering, the function then applies Top-p filtering. In this step, it looks at the cumulative probability of the sorted list of words (after Top-k filtering) and keeps the __smallest set__ of words whose cumulative __probability exceeds 'p'.__

In [18]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation.utils import top_k_top_p_filtering
import torch
from torch import nn

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

sequence = f"Hugging Face is based in DUMBO, New York City, and"
inputs = tokenizer(sequence, return_tensors="pt")
input_ids = inputs["input_ids"]

# get logits of last hidden state
next_token_logits = model(**inputs).logits[:, -1, :]

# filter potential next tokens
filtered_next_token_logits = top_k_top_p_filtering(next_token_logits, top_k=50, top_p=.7)

# sample next token
probs = nn.functional.softmax(filtered_next_token_logits, dim=-1)
next_token = torch.multinomial(probs, num_samples=1)

# concatenate sequence and prediction (PyTorch, dimensions are zero-indexed,
# so -1 refers to the last dimension. This means adding the new token to the end of the sequence.)
generated = torch.cat([input_ids, next_token], dim=-1)
resulting_string = tokenizer.decode(generated.tolist()[0])

print(resulting_string)

ImportError: cannot import name 'top_k_top_p_filtering' from 'transformers.generation.utils' (/usr/local/lib/python3.12/dist-packages/transformers/generation/utils.py)

This outputs a (hopefully) coherent next token following the original sequence, which in our case is the word **is** or
**features**.

You may run the code cell several times and check out different predictions.

In the next section, we show how `GenerationMixin.generate` can be used to
generate multiple tokens up to a specified length instead of one token at a time.

### Text Generation

In text generation (**a.k.a** **open-ended text generation**) the goal is to create a coherent portion of text that is a
continuation from the given context.


The following example shows how **GPT-2** can be used in pipelines to generate text. As a default all models apply **Top-K** sampling when used in pipelines, as configured in their respective configurations (see [gpt-2 config](https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-config.json) for example).

In [19]:
from transformers import pipeline
text_generator = pipeline("text-generation")
print(text_generator("Practical Data Science at the University of Wuerzburg is", max_length=50, do_sample=True))

No model was supplied, defaulted to openai-community/gpt2 and revision 607a30d (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'generated_text': 'Practical Data Science at the University of Wuerzburg is an online course that introduces you to the concept of "applied data science." Students will learn from data scientists who have worked in the academic field and from data science professors who have worked in the industry. The course is structured like a "software seminar," but students are encouraged to take some introductory data science courses before starting.\n\nStudents are asked to take two courses: Data Science Seminar and Applied Data Science Seminar. The course will be divided into two parts: Data Science and Data Science Seminar.\n\nThe data science course will take about 45 minutes to complete. It is a very straightforward course, but provides an interesting starting point for students to learn how to use the data science concepts that are commonly found in scientific research.\n\nStudents will also take a practical Data Science Seminar. This is a non-technical course where students use the data 

Here, the model generates a random text with a total maximal length of **50** tokens from context **"Practical Data Science at the University of Wuerzburg is"**. Behind the scenes, the pipeline object calls the method `PreTrainedModel.generate` to generate text. The default arguments for this method can be overridden in the pipeline, as is shown above for the arguments `max_length` and `do_sample`.

#### Model + Tokenizer

Below is an example of text generation using `XLNet` and its tokenizer, which includes calling `generate` directly:

In [20]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("xlnet-base-cased")
tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")

# Padding text helps XLNet with short prompts - proposed by Aman Rusia in https://github.com/rusiaaman/XLNet-gen#methodology
PADDING_TEXT = """In 1991, the remains of Russian Tsar Nicholas II and his family
(except for Alexei and Maria) are discovered.
The voice of Nicholas's young son, Tsarevich Alexei Nikolaevich, narrates the
remainder of the story. 1883 Western Siberia,
a young Grigori Rasputin is asked by his father and a group of men to perform magic.
Rasputin has a vision and denounces one of the men as a horse thief. Although his
father initially slaps him for making such an accusation, Rasputin watches as the
man is chased outside and beaten. Twenty years later, Rasputin sees a vision of
the Virgin Mary, prompting him to become a priest. Rasputin quickly becomes famous,
with people, even a bishop, begging for his blessing. <eod> </s> <eos>"""

prompt = "Today the weather is really nice and I am planning on "

inputs = tokenizer(PADDING_TEXT + prompt, add_special_tokens=False, return_tensors="pt")["input_ids"]
prompt_length = len(tokenizer.decode(inputs[0]))

outputs = model.generate(inputs, max_length=250, do_sample=True, top_p=0.95, top_k=60)
generated = prompt + tokenizer.decode(outputs[0])[prompt_length+1:]
print(generated)

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/467M [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

This is a friendly reminder - the current text generation call has exceeded the model's predefined maximum length (-1). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


Today the weather is really nice and I am planning on coming back to the house. On Sunday I will be back in the van with friends. In the morning I would be going to a local coffee shop in this place.<eop> I will be driving the van around a corner and doing some crazy walking. There are lots of bushes around, in the middle of the road, and to a huge tree. There is a lot


Text generation is currently possible with *GPT-2*, *OpenAi-GPT*, *CTRL*, *XLNet*, *Transfo-XL* and *Reformer* in
PyTorch and for most models in Tensorflow as well.

As can be seen in the example above *XLNet* and *Transfo-XL* often need to be __padded to work well.__ By adding a padding text, you're essentially giving the model a broader context to 'understand' before it starts generating text based on your actual prompt.

GPT-2 is usually a good choice for **open-ended text generation** because it was trained on millions of webpages with a causal language modeling objective.

For more information on how to apply different decoding strategies for text generation, please also refer to this text
generation blog post [here](https://huggingface.co/blog/how-to-generate).

### Named Entity Recognition

Named Entity Recognition (NER) is the task of __classifying tokens__ according to a class, for example, identifying a token
as a person, an organisation or a location.

Here is an __example__ of using pipelines to do named entity recognition, specifically, trying to identify tokens as
belonging to one of __9 classes:__

- O, Outside of a named entity
- B-MIS, Beginning of a miscellaneous entity right after another miscellaneous entity
- I-MIS, Miscellaneous entity
- B-PER, Beginning of a person's name right after another person's name
- I-PER, Person's name
- B-ORG, Beginning of an organisation right after another organisation
- I-ORG, Organisation
- B-LOC, Beginning of a location right after another location
- I-LOC, Location

It leverages a fine-tuned model on [CoNLL-2003](https://paperswithcode.com/dataset/conll-2003), fine-tuned by [@stefan-it](https://github.com/stefan-it) from [dbmdz](https://github.com/dbmdz).

In [21]:
from transformers import pipeline
ner_pipe = pipeline("ner")
sequence = """Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO,
therefore very close to the Manhattan Bridge which is visible from the window."""

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cuda:0


This outputs a list of all words that have been identified as one of the entities from the 9 classes defined above.
Here are the expected results:

In [22]:
for entity in ner_pipe(sequence):
    print(entity)

{'entity': 'I-ORG', 'score': np.float32(0.99957865), 'index': 1, 'word': 'Hu', 'start': 0, 'end': 2}
{'entity': 'I-ORG', 'score': np.float32(0.9909764), 'index': 2, 'word': '##gging', 'start': 2, 'end': 7}
{'entity': 'I-ORG', 'score': np.float32(0.9982224), 'index': 3, 'word': 'Face', 'start': 8, 'end': 12}
{'entity': 'I-ORG', 'score': np.float32(0.9994879), 'index': 4, 'word': 'Inc', 'start': 13, 'end': 16}
{'entity': 'I-LOC', 'score': np.float32(0.9994344), 'index': 11, 'word': 'New', 'start': 40, 'end': 43}
{'entity': 'I-LOC', 'score': np.float32(0.99931955), 'index': 12, 'word': 'York', 'start': 44, 'end': 48}
{'entity': 'I-LOC', 'score': np.float32(0.9993794), 'index': 13, 'word': 'City', 'start': 49, 'end': 53}
{'entity': 'I-LOC', 'score': np.float32(0.98625827), 'index': 19, 'word': 'D', 'start': 79, 'end': 80}
{'entity': 'I-LOC', 'score': np.float32(0.951427), 'index': 20, 'word': '##UM', 'start': 80, 'end': 82}
{'entity': 'I-LOC', 'score': np.float32(0.93365926), 'index': 21, 

Note how the tokens of the sequence "Hugging Face" have been identified as an organisation, and "New York City",
"DUMBO" and "Manhattan Bridge" have been identified as locations.


#### Model + Tokenizer


Here is an example of doing named entity recognition, using a model and a tokenizer. The process is the following:

1. Instantiate a tokenizer and a model from the checkpoint name. The model is identified as a BERT model and loads it
   with the weights stored in the checkpoint.
2. Define a sequence with known entities, such as "Hugging Face" as an organisation and "New York City" as a location.
3. Split words into tokens so that they can be mapped to predictions. We use a small hack by, first, completely
   encoding and decoding the sequence, so that we're left with a string that contains the special tokens.
4. Encode that sequence into IDs (special tokens are added automatically).
5. Retrieve the predictions by passing the input to the model and getting the first output. This results in a
   distribution over the 9 possible classes for each token. We take the argmax to retrieve the most likely class for
   each token.
6. Zip together each token with its prediction and print it.

In [23]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = "Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO, " \
           "therefore very close to the Manhattan Bridge."
inputs = tokenizer(sequence, return_tensors="pt")
tokens = inputs.tokens()

outputs = model(**inputs).logits
predictions = torch.argmax(outputs, dim=2)

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [24]:
for token, prediction in zip(tokens, predictions[0].numpy()):
    print((token, model.config.id2label[prediction]))

('[CLS]', 'O')
('Hu', 'I-ORG')
('##gging', 'I-ORG')
('Face', 'I-ORG')
('Inc', 'I-ORG')
('.', 'O')
('is', 'O')
('a', 'O')
('company', 'O')
('based', 'O')
('in', 'O')
('New', 'I-LOC')
('York', 'I-LOC')
('City', 'I-LOC')
('.', 'O')
('Its', 'O')
('headquarters', 'O')
('are', 'O')
('in', 'O')
('D', 'I-LOC')
('##UM', 'I-LOC')
('##BO', 'I-LOC')
(',', 'O')
('therefore', 'O')
('very', 'O')
('close', 'O')
('to', 'O')
('the', 'O')
('Manhattan', 'I-LOC')
('Bridge', 'I-LOC')
('.', 'O')
('[SEP]', 'O')


This outputs a list of each token mapped to its corresponding prediction. Differently from the pipeline, here every
token has a prediction as we didn't remove the "0"th class, which means that no particular entity was found on that
token.

In the above example, `predictions` is an integer that corresponds to the predicted class. We can use the
`model.config.id2label` property in order to recover the class name corresponding to the class number.

### Summarization

Summarization is the task of summarizing a document or an article __into a shorter text.__

Here is an example of using the pipelines to do summarization. It leverages a Bart model that was fine-tuned on the CNN
/ Daily Mail data set.

In [25]:
from transformers import pipeline

summarizer = pipeline("summarization")

ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [26]:
print(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False))

[{'summary_text': ' Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002 . At one time, she was married to eight men at once, prosecutors say .'}]


Because the summarization pipeline depends on the `PreTrainedModel.generate()` method, we can override the default
arguments of `PreTrainedModel.generate()` directly in the pipeline for `max_length` and `min_length`.

#### Model + Tokenizer

Here is an example of doing summarization using a model and a tokenizer. The process is the following:

1. Instantiate a tokenizer and a model from the checkpoint name. Summarization is usually done using an encoder-decoder
   model, such as `Bart` or `T5`.
2. Define the article that should be summarized.
3. Add the T5 specific prefix "summarize: ".
4. Use the `PreTrainedModel.generate()` method to generate the summary.

In this example we use Google's T5 model. Even though it was pre-trained only on a multi-task mixed dataset (including
CNN / Daily Mail), it yields very good results.

In [27]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

# T5 uses a max_length of 512 so we cut the article to 512 tokens.
inputs = tokenizer("summarize: " + ARTICLE, return_tensors="pt", max_length=512, truncation=True)

outputs = model.generate(
    inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True
)
print(tokenizer.decode(outputs[0]))

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

<pad> prosecutors say the marriages were part of an immigration scam . if convicted, barrientos faces two criminal counts of "offering a false instrument for filing in the first degree" she has been married 10 times, nine of them between 1999 and 2002 .</s>


### Translation

Translation is the task of translating a text from one language to another.


An example of a translation dataset is the __WMT English to German__ dataset, which has sentences in English as the input
data and the corresponding sentences in German as the target data.

Here is an example of using the pipelines to do translation. It leverages a T5 model that was only pre-trained on a
multi-task mixture dataset (including WMT), yet, yielding impressive translation results.

In [28]:
from transformers import pipeline

translator = pipeline("translation_en_to_de")

print(translator("Hugging Face is a technology company based in New York and Paris", max_length=40))

No model was supplied, defaulted to google-t5/t5-base and revision a9723ea (https://huggingface.co/google-t5/t5-base).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=40) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'translation_text': 'Hugging Face ist ein Technologieunternehmen mit Sitz in New York und Paris.'}]


Because the translation pipeline depends on the `PreTrainedModel.generate()` method, we can override the default
arguments of `PreTrainedModel.generate()` directly in the pipeline as is shown for `max_length` above.


#### Model + Tokenizer

Here is an example of doing translation using a model and a tokenizer. The process is the following:

1. Instantiate a tokenizer and a model from the checkpoint name. Summarization is usually done using an encoder-decoder
   model, such as `Bart` or `T5`.
2. Define the article that should be summarized.
3. Add the T5 specific prefix "translate English to German: "
4. Use the `PreTrainedModel.generate()` method to perform the translation.

In [29]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")
inputs = tokenizer(
    "translate English to German: Hugging Face is a technology company based in New York and Paris",
    return_tensors="pt"
)
outputs = model.generate(inputs["input_ids"], max_length=40, num_beams=4, early_stopping=True)
print(tokenizer.decode(outputs[0]))

<pad> Hugging Face ist ein Technologieunternehmen mit Sitz in New York und Paris.</s>


## Fine-tuning a pretrained model

In this section, we will fine-tune a pretrained model from the Transformers library. In PyTorch, there is no generic training loop so
the 🤗 Transformers library provides an API with the class [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) to let you fine-tune or train
a model from scratch easily. Then we will show you how to alternatively write the whole training loop in PyTorch.

Before we can fine-tune a model, we need a dataset. In this tutorial, we will show you how to fine-tune BERT on the
[IMDB dataset](https://www.imdb.com/interfaces/): the task is to classify whether movie reviews are positive or
negative. For examples of other tasks, refer to the final part of this notebooks, where we provide additional resources.

<a id='data-processing'></a>

### Load dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets/) library to download and preprocess the IMDB
datasets. We will go over this part pretty quickly. Since the focus of this tutorial is on training, you should refer
to the 🤗 Datasets [documentation](https://huggingface.co/docs/datasets/) or the [preprocessing](https://huggingface.co/docs/transformers/master/en/preprocessing) tutorial for
more information.

First, we can use the `load_dataset` function to download and cache the dataset:

In [30]:
from datasets import load_dataset

raw_datasets = load_dataset("imdb")

README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

This works like the `from_pretrained` method we saw for the models and tokenizers (except the cache directory is
_~/.cache/huggingface/dataset_ by default).

The `raw_datasets` object is a dictionary with three keys: `"train"`, `"test"` and `"unsupervised"`
(which correspond to the three splits of that dataset). We will use the `"train"` split for training and the
`"test"` split for validation.

To preprocess our data, we will need a tokenizer:

In [31]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

As we saw in [preprocessing](https://huggingface.co/docs/transformers/master/en/preprocessing), we can prepare the text inputs for the model with the following command (this is an
example, not a command you can execute):

```python
inputs = tokenizer(sentences, padding="max_length", truncation=True)
````

This will make all the samples have the maximum length the model can accept (here 512), either by padding or truncating
them.

However, we can instead apply these preprocessing steps to all the splits of our dataset at once by using the
`map` method:

In [32]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

You can learn more about the map method or the other ways to preprocess the data in the 🤗 Datasets [documentation](https://huggingface.co/docs/datasets/).

Next we will generate a small subset of the training and validation set, to enable faster training:

In [33]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

In all the examples below, we will always use `small_train_dataset` and `small_eval_dataset`. Just replace
them by their _full_ equivalent to train or evaluate on the full dataset.

<a id='trainer'></a>

### Exercises - Exploring Huggingface Models

There are many public pretrained models hosted on huggingface. Check out this link: https://huggingface.co/models.

1. Try to filter models in "Question Answering" category, search for the Bilingual English + German SQuAD2.0 model of Deutsche Telekom (Hint: it should be in the first page after filtering).

2. Read through the document, try to load the model using `transformers` library.

3. Feed the below QA input into the model and check for the result. Feel free to play around with the input.

Sometimes the models are not *public*, but *private* or *gated*. In this case, you need to create a token (with a registered account) and pass it to Google Colab. You don't need it for these exercises. More details will be shown in the tutorial.

In [34]:
from transformers import pipeline

context_de = """
Die Stadt München ist bekannt für ihre beeindruckende Architektur und
reiche Kultur. Besonders berühmt ist das Oktoberfest,
das jedes Jahr Millionen von Besuchern anzieht,
die die bayerische Tradition und die köstlichen Biere genießen möchten.
Neben dem Fest bietet München zahlreiche Museen, wie die Alte Pinakothek,
die eine der bedeutendsten Sammlungen europäischer Malerei beherbergt.
"""
context_en = """
In the heart of a bustling city, a small bookstore named "The Enchanted Page"
has become a sanctuary for book lovers. With its cozy reading nooks and shelves
brimming with both classic literature and contemporary bestsellers, it offers
a unique escape from the urban chaos. Every Saturday, the store hosts a book club
where local authors share their insights and readers discuss their
favorite novels over coffee.
"""

question_de = "Was ist das Oktoberfest und warum lockt es so viele Besucher nach München?"
question_en = 'What does "The Enchanted Page" bookstore offer to engage the community on Saturdays?'

# YOUR CODE HERE
# create a pipeline with <task name>, <model> and <tokenizer>
# hint: model and tokenizer receives the same model name
qa_pipeline = pipeline(
    "task-name",
    model=...,
    tokenizer=...
)

# predict
qa_pipeline(context=..., question=...)

KeyError: "Unknown task task-name, available tasks are ['audio-classification', 'automatic-speech-recognition', 'depth-estimation', 'document-question-answering', 'feature-extraction', 'fill-mask', 'image-classification', 'image-feature-extraction', 'image-segmentation', 'image-text-to-text', 'image-to-image', 'image-to-text', 'keypoint-matching', 'mask-generation', 'ner', 'object-detection', 'question-answering', 'sentiment-analysis', 'summarization', 'table-question-answering', 'text-classification', 'text-generation', 'text-to-audio', 'text-to-speech', 'text2text-generation', 'token-classification', 'translation', 'video-classification', 'visual-question-answering', 'vqa', 'zero-shot-audio-classification', 'zero-shot-classification', 'zero-shot-image-classification', 'zero-shot-object-detection', 'translation_XX_to_YY']"

### Fine-tuning in PyTorch with the Trainer API

Since PyTorch does not provide a training loop, the 🤗 Transformers library provides a [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer)
API that is optimized for 🤗 Transformers models, with a wide range of training options and with built-in features like
logging, gradient accumulation, and mixed precision.

First, let's define our model:

In [35]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


This will issue a __warning__ about some of the __pretrained weights not being__ used and some __weights being randomly
initialized:__
* We are throwing away the __pretraining head of the BERT model__ and
* __replace__ it with a classification __head__ which is __randomly initialized.__


We will fine-tune this model on our task, transferring the knowledge of the pretrained model to it (which is why doing this is __called transfer learning).__

### Trainer and Training Arguments



Then, to define our __[Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer),__ we will need to instantiate
__[TrainingArguments](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.TrainingArguments).__



This class contains all the __hyperparameters__ we can tune for the [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) or the flags to activate the different __training options__ it supports.

Let's begin by using all the defaults, the only thing we then have to provide is a directory in which the checkpoints will be saved:

In [36]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")

Then we can instantiate a [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) like this:

In [37]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset
)

To fine-tune our model, we just need to call

In [38]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: gunther-gust (gunther-gust-university-of-wuerzburg) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


TrainOutput(global_step=375, training_loss=0.36385693359375, metrics={'train_runtime': 654.7856, 'train_samples_per_second': 4.582, 'train_steps_per_second': 0.573, 'total_flos': 789333166080000.0, 'train_loss': 0.36385693359375, 'epoch': 3.0})

which will start a training that you can follow with a progress bar, which should take a couple of minutes to complete
(as long as you have access to a GPU). It won't actually tell you anything useful about how well (or badly) your model
is performing however as by default, there is no evaluation during training, and we didn't tell the
[Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) to compute any metrics. Let's have a look on how to do that now!



## Add performance metrics

To have the [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) compute and report metrics, we need to give it a `compute_metrics`
function that takes predictions and labels (grouped in a namedtuple called [EvalPrediction](https://huggingface.co/docs/transformers/master/en/internal/trainer_utils#transformers.EvalPrediction)) and
return a dictionary with string items (the metric names) and float values (the metric values).

The 🤗 `evaluate` library provides an easy way to get the common metrics used in NLP with the __`load` function.__


Here we simply use accuracy. Then we define the `compute_metrics` function that just convert logits to predictions
(remember that all 🤗 Transformers models return the logits) and feed them to `compute` method of this metric.

In [39]:
# install evaluate
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.9 MB/s eta 0:00:00


In [40]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

The compute function needs to receive a tuple (with logits and labels) and has to return a dictionary with string keys
(the name of the metric) and float values. It will be called at the end of each evaluation phase on the whole arrays of
predictions/labels.

To check if this works on practice, let's create a new [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) with our fine-tuned model:

In [41]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

{'eval_loss': 0.5343369245529175,
 'eval_model_preparation_time': 0.0089,
 'eval_accuracy': 0.876,
 'eval_runtime': 34.7056,
 'eval_samples_per_second': 28.814,
 'eval_steps_per_second': 3.602}

which showed an accuracy of 87.5% in our case.

If you want to fine-tune your model and __regularly report the evaluation metrics__ (for instance at the end of each
__epoch__), here is how you should define your training arguments:

In [45]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer", eval_strategy="epoch")

See the documentation of [TrainingArguments](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.TrainingArguments) for more options.


<a id='keras'></a>

### Additional resources

To look at more fine-tuning examples you can refer to:

- [🤗 Transformers Examples](https://github.com/huggingface/transformers/tree/master/examples) which includes scripts
  to train on all common NLP tasks in PyTorch and TensorFlow.

- [🤗 Transformers Notebooks](https://huggingface.co/docs/transformers/master/en/notebooks) which contains various notebooks and in particular one per task (look for
  the _how to finetune a model on xxx_).
  
- [Fastai Notebooks](https://docs.fast.ai/tutorial.transformers.html) An example of how to incorporate the transfomers library from HuggingFace with fastai

# Mentimeter


<img src="https://github.com/GuntherGust/tds2_data/blob/main/images/d3.png?raw=true" style="width:50%; float:center;" />
